# utils

> This module includes utility functions that are used across multiple modules, such as coordinate transformations, interpolation functions, and statistical analysis tools.

In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [2]:
#| export
from LensCalcPy.parameters import *
import numpy as np

In [6]:
#| export

def density_nfw(r: float, # distance to MW center in kpc
                ) -> float: # DM density in Msun/kpc^3
    return rhoc / ((r/rs) * (1 + r/rs)**2)

def mass_enclosed_nfw(r: float  # distance to MW center in kpc
                      ) -> float : # enclosed DM mass in Msun
    return 4*np.pi * rhoc * rs**3 * (np.log(1 + r/rs) - (r/rs)/(1 + r/rs))

def dist(d: float # distance from the Sun in kpc
         ) -> float: #weighted lensing distance in kpc
    return d * (1 - d/ds)

def einstein_rad(d: float, # distance from the Sun in kpc
                 mass: float, # mass of the lens in Msun
                 ) -> float:
    return np.real((4 * G * mass * dist(d)/c**2)**(1/2))

def dist_mw(d: float, # distance from the Sun in kpc
            ) -> float: #distance to the MW center in kpc
    return np.sqrt(d**2 + rEarth**2 - 2*d*rEarth*np.cos(np.radians(l))*np.cos(np.radians(b)))

def velocity_dispersion(d: float, # distance from the Sun in kpc
                        ) -> float: # velocity dispersion in km/s
    r = dist_mw(d)
    return np.sqrt(G * mass_enclosed_nfw(r) / r) 

def velocity_radial(d: float, # distance from the Sun in kpc
                    mass: float, # mass of the lens in Msun
                    umin: float, # minimum impact parameter
                    t: float, # crossing time in hours
                    ) -> float: # radial velocity in km/s
    return 2*einstein_rad(d, mass) * (ut**2 - umin**2)**(1/2) / t * kpctokm

# from below 16 of https://iopscience.iop.org/article/10.3847/1538-4357/ac07a8/pdf*)
# alphabar = 27 Degrees xp-axis is aligned with the major axis
# of the Galactic bar,where \[Alpha]bar=27\[Degree] is applied as the bar angle.
# assuming z ~= 0 and we are looking at line of sight to galactic center from Sun
# galactocentric coordniates x', y' as function of d, distance from Sun

def get_primed_coords(d: float # distance from Sun
                      )-> tuple:
    """Get galactocentric coordinates x', y' as function of d, distance from Sun
    """
    x = rEarth - d
    y = 0
    return (x**2 + y**2)**0.5 * np.cos(alphabar*np.pi/180), (x**2 + y**2)**0.5 * np.sin(alphabar*np.pi/180)

def scientific_format(x, pos):
    """
    Formats a number in scientific notation in latex
    """
    a, b = '{:.1e}'.format(x).split('e')
    b = int(b)
    return r'${} \times 10^{{{}}}$'.format(a, b)

In [ ]:
#| export

# Disk Density
def zthin(r):
    if r > 4.5:
        return zthinSol - (zthinSol - zthin45) * (rsol - r) / (rsol - 4.5)
    else:
        return zthin45

def rho_thin(r, z):
    if r > rdBreak:
        return rho_thin_Sol * zthinSol / zthin(r) * \
            np.exp(-((r - rsol) / rthin)) * (1 / np.cos(np.abs(z) / zthin(r)))**2
    else:
        return rho_thin_Sol * zthinSol / zthin(r) * \
            np.exp(-((rdBreak - rsol) / rthin)) * (1 / np.cos(np.abs(z) / zthin(r)))**2

def rho_thick(r, z):
    if r > rdBreak:
        return rho_thick_Sol * np.exp(-((r - rsol) / rthick)) * \
            np.exp(-(np.abs(z) / zthickSol))
    else:
        return rho_thick_Sol * np.exp(-((rdBreak - rsol) / rthick)) * \
            np.exp(-(np.abs(z) / zthickSol))

# Bulge Density
def rsf(xp, yp, zp):
    R = (xp**cperp / x0**cperp + yp**cperp / y0**cperp)**(cpar/cperp) + (zp / z0)**cpar
    return R**(1/cpar)

def fE(xp, yp, zp):
    return np.exp(-rsf(xp, yp, zp))

def cut(x):
    if x > 0:
        return np.exp(-x**2)
    else:
        return 1

def rho_bulge(xp, yp, zp):
    R = (xp**2 + yp**2 + zp**2)**0.5
    return rho0_B * fE(xp, yp, zp) * cut((R - Rc) / 0.5)

def rho_bulge(d):
    xp, yp = get_primed_coords(d)
    zp = 0
    R = (xp**2 + yp**2 + zp**2)**0.5
    return rho0_B * fE(xp, yp, zp) * cut((R - Rc) / 0.5)

# Total FFP Density
def rho_FFPs(d: float, # distance from Sun in kpc
             ) -> float: # FFP density in Msun/kpc^3
    #! Check units here
    r = dist_mw(d)
    z = 0
    return rho_thin(r, z) + rho_thick(r, z) + rho_bulge(d)



In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()